In [1]:
import pandas as pd
#from lxml import etree
import requests
from bs4 import BeautifulSoup

In [2]:
import json
import unicodedata
from datetime import datetime

---- using HTML response ----

In [3]:
import locale
locale.getdefaultlocale()

('sv_SE', 'UTF-8')

In [4]:
#locale.setlocale(locale.LC_TIME, "sv_SE.UTF-8")
#locale.setlocale(locale.LC_ALL, 'sv_SE')
locale.setlocale(locale.LC_TIME, "sv_SE")
#locale.setlocale(locale.LC_ALL, '')

'sv_SE'

In [5]:
url = "https://www.hemnet.se/salda/bostader"
params= {
    #'housing_form_groups':'apartments',
    'location_ids':'17853',
    #'item_types':'bostadsratt',
    'by':'sale_date', #sort by sold date
    'order':'asc',
    'preferred_sorting':'true',
    'sold_age':'6m', #1d,4d,1w,1m and so on...
    'page':1,
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
soup = BeautifulSoup(response.content, "html.parser")
response.url

'https://www.hemnet.se/salda/bostader?location_ids=17853&by=sale_date&order=asc&preferred_sorting=true&sold_age=6m&page=1'

In [16]:
#list to add our objects to
item_list=[]

def read_pagedata(soupref):
    #find the ul with the results list
    result_ul=soupref.find('ul','sold-results')

    #get the list items with our data (filter out the ads)
    result_li=result_ul('li', 'sold-results__normal-hit')

    #print(result_li)

    #start loop...
    for item in result_li:
        #a dict to collect all key/values in
        item_dict={}

        #get tracking data json for ids...
        tracking_data=item.attrs['data-tracking-data']
        tracking_data_json=json.loads(tracking_data)

        #add ids
        item_dict['listingId']=tracking_data_json['listingId']
        item_dict['saleId']=tracking_data_json['saleId']
        
        #get the anchor href for the url
        result_li_a=item.find('a','sold-property-link')
        item_dict['url']=result_li_a['href']

        #get our sale listing info
        result_price_info=result_li_a.find('div','sold-property-listing__location')

        #address
        address=result_li_a.find('h2','sold-property-listing__heading').string.strip()
        #address=address.replace(' / ','/') #unicode getting in the way again?
        item_dict['address']=address

        #fee
        fee_el=result_li_a.find('div','sold-property-listing__fee')
        if fee_el:
            fee=fee_el.string.strip()
            fee=int("".join(unicodedata.normalize("NFKD",fee).strip('kr/mån').split()))
            item_dict['fee']=fee

        #living area
        #area=result_li_a.find('div','sold-property-listing__area').string.strip()
        #area=int("".join(unicodedata.normalize("NFKD",fee).strip('kr/mån').split()))
        #item_dict['area']=area
        area_el=result_li_a.find('div','sold-property-listing__area')
        if area_el:
            area=area_el.text.strip()
            area_arr="".join(area.split('m²')).split()
            
            print(area_arr)

            span_el = result_li_a.find('span', 'listing-card__attribute--normal-weight')
            if span_el: 
                total_area=\
                    float(area_arr[0].replace(",", ".")) + \
                    float(area_arr[2].replace(",", "."))
                item_dict['living_area']=total_area
                if len(area_arr) > 4:
                    item_dict['rooms']=area_arr[3]
            else:
                item_dict['living_area']=area_arr[0]
                if len(area_arr) > 1:
                    item_dict['rooms']=area_arr[1] 
        
        #get our sale price info
        result_price_info=result_li_a.find('div','sold-property-listing__price-info')

        #ending date
        end_date_el=result_price_info.find('div',class_="sold-property-listing__sold-date")
        if end_date_el:
            end_date_str=end_date_el.string.strip()
            end_date_str=end_date_str.replace('Såld ','')
            end_date=datetime.strptime(end_date_str, '%d %B %Y')
            item_dict['end_date']=end_date

        #end price
        end_price_el=result_price_info.find('div',class_="sold-property-listing__subheading")
        if end_price_el:
            end_price_str=end_price_el.string.strip()
            end_price=int("".join(end_price_str.strip(' kr').replace('Slutpris ','').split()))
            item_dict['end_price']=end_price

        #end price/m2
        end_price_area_el=result_price_info.find('div',class_="sold-property-listing__price-per-m2")
        if end_price_area_el:
            end_price_area_str=end_price_area_el.string.strip()
            end_price_area=int("".join(unicodedata.normalize("NFKD",end_price_area_str).strip('kr/m2').split()))
            item_dict['end_price_area']=end_price_area
        
        #price change
        end_price_change_el=result_price_info.find('div',class_="sold-property-listing__price-change")
        if end_price_change_el:
            end_price_change_str=end_price_change_el.string.strip()
            end_price_change_str="".join(end_price_change_str.strip(' %').split())
            end_price_change=int(end_price_change_str[1:])
            end_price_change_sign=end_price_change_str[0]

            #calculate rate (percentage in float)
            if end_price_change !=0:
                item_dict['end_price_change_rate']=eval(f'100{end_price_change_sign}{end_price_change}')/100
            elif end_price_change==0:
                item_dict['end_price_change_rate']=100/100

            item_dict['end_price_change']=end_price_change
            item_dict['end_price_change_sign']=end_price_change_sign

        item_list.append(item_dict)

#print(item_list)

In [17]:
read_pagedata(soup)
next_link=soup.find('a','next_page')
while next_link:
    print(next_link['href'])
    next_response = requests.request("GET", "https://www.hemnet.se"+next_link['href'], headers=headers)
    next_soup = BeautifulSoup(next_response.content, "html.parser")
    read_pagedata(next_soup)
    next_link=next_soup.find('a','next_page')

['35', '2', 'rum']
['154', '6', 'rum']
['178', '+', '18', '7', 'rum']
['86,5', '4', 'rum']
['58', '2', 'rum']
['81', '3', 'rum']
['63', '2', 'rum']
['43,6', '1,5', 'rum']
['87,8', '4', 'rum']
['91,5', '4', 'rum']
['73,5', '3', 'rum']
['63', '2', 'rum']
['92', '3,5', 'rum']
['60', '3', 'rum']
['43', '2', 'rum']
['37', '1,5', 'rum']
['63', '2,5', 'rum']
['69', '2', 'rum']
['81', '4', 'rum']
['46', '2', 'rum']
['122', '5', 'rum']
['90', '4', 'rum']
['61', '3', 'rum']
['61', '2', 'rum']
['61', '2', 'rum']
['38', '1', 'rum']
['61', '2', 'rum']
['72', '3', 'rum']
['44', '2', 'rum']
['206', '6', 'rum']
['19', '1', 'rum']
['59,5', '2,5', 'rum']
['40', '2', 'rum']
['93', '3', 'rum']
['91', '3', 'rum']
['115', '5', 'rum']
['86,3', '3', 'rum']
['64', '2', 'rum']
['53', '2', 'rum']
['65', '2', 'rum']
['38', '1', 'rum']
['79', '3', 'rum']
['81,5', '3', 'rum']
['21,5', '1', 'rum']
['31', '1', 'rum']
['58', '2,5', 'rum']
['41', '2', 'rum']
['115', '4', 'rum']
['37', '1,5', 'rum']
['2', 'rum']
/salda/

['63', '2', 'rum']
['129', '+', '54', '5', 'rum']
['41', '2', 'rum']
['128', '+', '89', '7', 'rum']
['71,5', '2', 'rum']
['38', '2', 'rum']
['67', '2', 'rum']
['37', '1,5', 'rum']
['59,5', '2', 'rum']
['175', '6', 'rum']
['29,5', '1', 'rum']
['29,5', '1', 'rum']
['53', '2', 'rum']
['90', '4', 'rum']
['208', '+', '62', '6', 'rum']
['90', '3', 'rum']
['113', '4', 'rum']
['71', '4', 'rum']
['102', '4', 'rum']
['53', '2', 'rum']
['86,3', '3', 'rum']
['71', '2', 'rum']
['77,8', '3', 'rum']
['77,8', '3,5', 'rum']
['52', '2', 'rum']
['85', '3', 'rum']
['93', '4', 'rum']
['58,5', '2', 'rum']
['55', '2', 'rum']
['109', '3', 'rum']
['162', '+', '10', '7', 'rum']
['50', '2', 'rum']
['79,5', '3', 'rum']
['121', '+', '60', '7', 'rum']
['81', '3,5', 'rum']
['59', '2', 'rum']
['135', '+', '117', '6', 'rum']
['28,1', '2', 'rum']
['98', '4', 'rum']
['90', '5', 'rum']
['37', '1', 'rum']
['96', '3', 'rum']
['118', '5', 'rum']
['62,5', '2', 'rum']
['65', '2', 'rum']
['176', '+', '9', '6', 'rum']
['66,2', 

IndexError: list index out of range

In [ ]:
df=pd.json_normalize(item_list)
df.info()

In [ ]:
df

In [ ]:
df[df["fee"].isna()]

import json
import unicodedata
from datetime import datetime


#find the ul with the results list
result_ul=soup.find('ul','sold-results')

#get the list items with our data (filter out the ads)
result_li=result_ul('li', 'sold-results__normal-hit')

print(len(result_li))

#start loop...
for item in result_li:
    
    #get the anchor href for the url
    result_li_a=item.find('a','sold-property-link')

    #living area
    area=result_li_a.find('div','sold-property-listing__area').string.strip()
    area_arr="".join(area.split('m²')).split()
    rooms=area_arr[1]
    living_area=area_arr[0]
    #area=int("".join(unicodedata.normalize("NFKD",area).strip('kr/mån').split()))
    #address
    #address_str=result_price_info.find('div',class_='sold-property-listing__location').string.strip()
    #print(address_str)